# 1. Import Environment Variables


In [4]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

# 2. Connect to OPENAI (or OLLAMA or GEMINI)

The next cell is where we load in the environment variables and connect to the OpenAI API. Make sure you have your API key stored in an environment variable named `OPENAI_API_KEY`.

```pythonimport os
import openai

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
```


In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-. Please recheck your API key.")
else:
    print("API key found!")

API key found!


# Let's test the API key


In [6]:
message = """
    Hello, GPT! This is my first ever message to you! I am learning AI engineering
    and I am thoroughly excited to see what is coming ahead!
"""

messages = [{"role": "user", "content": message}]

messages

[{'role': 'user',
  'content': '\n    Hello, GPT! This is my first ever message to you! I am learning AI engineering\n    and I am thoroughly excited to see what is coming ahead!\n'}]

In [7]:
# This is just generic OPENAI API response, save it somewhere

openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

'Welcome! That’s awesome to hear. AI engineering is a fantastic area with lots to learn and build. I’d love to tailor help to your goals. A couple quick questions to start:\n\n- What language and stack are you most comfortable with? (Python is the usual starting point.)\n- Are you more interested in ML theory, practical model deployment, data engineering, or end-to-end ML systems (MLOps)?\n- Do you prefer quick hands-on projects or structured coursework?\n\nIf you want, we can jump into a small starter project right away. Here’s a simple, hands-on plan you can complete in a weekend to get a feel for building and deploying a model:\n\nStarter project: Train a classifier and serve it via an API\n- Goal: Build a small ML classifier (e.g., Iris or Titanic dataset) and expose a prediction API.\n- Tech stack (recommended): Python, scikit-learn, FastAPI, Docker (optional for packaging).\n- Steps:\n  1) Prepare your environment: create a Python virtual environment and install scikit-learn, fas

## Let's try creating our first project


In [8]:
# Let's try out this utility

# You can use any website of your choice below
albert = fetch_website_contents("https://www.alberteinstein.com/")
print(albert)

Albert Einstein – The Official Website of Albert Einstein

Skip to content
Biography
Publications
Awards & Honors
Historical Photo Gallery
Licensing
Programs
Menu
Close
Biography
Publications
Awards & Honors
Historical Photo Gallery
Licensing
Programs
Albert Einstein
Biography
The embodiment of genius and the pre-eminent scientist of the modern age, his theories and discoveries have profoundly affected the way people view and understand the world and their place in it. Einstein was also known as a philosopher and humanist who was keenly interested in and concerned about the affairs of the world.
His sagacious, wise, and humorous quotations, letters, and articles are widely used throughout popular culture as well as in historical and academic works. Einstein’s name and image are instantly recognizable everywhere in the world.
Albert Einstein was a theoretical physicist and the most famous scientist in human history. He developed the general theory of relativity, one of the two pillars o

## Types of Prompts

Models like GPT have been trained to receive instructions in a particular way.

- **A system prompt**: This sets the behavior of the model. For example, "You are a helpful assistant."

- **A user prompt**: This is the main instruction or question from the user. For example, "Explain the theory of relativity."

- **An assistant prompt**: This is the model's response to the user prompt.


In [9]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a helpful assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [10]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.
"""

## Messages

The API from OpenAI expects to receive messages in a specific format, which is a list of dictionaries. Each dictionary represents a message and contains two keys: `role` and `content`.

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```

In [11]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)
response.choices[0].message.content

'2 + 2 equals 4.'

## For reusability, we can define a function to create messages

In [12]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [13]:
messages_for(albert)

[{'role': 'system',
  'content': '\nYou are a helpful assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\nAlbert Einstein – The Official Website of Albert Einstein\n\nSkip to content\nBiography\nPublications\nAwards & Honors\nHistorical Photo Gallery\nLicensing\nPrograms\nMenu\nClose\nBiography\nPublications\nAwards & Honors\nHistorical Photo Gallery\nLicensing\nPrograms\nAlbert Einstein\nBiography\nThe embodiment of genius and the pre-eminent scientist of the modern age, his theories and discoveries have profoundly affected the way people view and understand the world and their place in it. Einstein was also known a

In [14]:
# And now: call the OpenAI API

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://www.alberteinstein.com/")

'Ah, the Official Albert Einstein website – basically your one-stop shop for soaking up the genius of the guy who made physics cool (and complicated). Dive into his biography where they remind you he was more than just a wild-haired math wizard; he was also a philosopher, humanist, and lover of witty quotes that still get reposted more than your favorite memes. Expect a deep dive into his groundbreaking science like general relativity and E=mc², along with a gallery of classic photos proving he really did wear those iconic looks. No breaking news because, well, he’s frozen in time, but the site is perfect for geeking out on the life and legacy of the man who made us question space, time, and probably our sanity.'

## For readability, we can show the messages in a nicer format as markdown

In [16]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://www.alberteinstein.com/")

Welcome to the official shrine of Albert Einstein – the guy who basically rewired our understanding of the universe and made physics look cool. Here, you'll find deep dives into his life, mind-blowing publications, endless awards (because clearly one Nobel Prize wasn't enough), and a photo gallery proving he rocks the wild hair look like no other. No breaking news or earth-shattering announcements, just the timeless legacy of the ultimate genius, philosopher, and part-time humanist who gave us E=mc² and countless wisecracks for the ages. Prepare to feel both smart and slightly inadequate.

<table style="margin: 0; text-align: left;">
    <tr>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI
# response =

# Step 4: print the result
# print(